In [14]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader
import datetime as dt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import display, HTML
import plotly.express as px

In [15]:
def historical_exchage_rate_provider_from_yahoo_finance(
    list_of_currencies= ['USD','TWD','AUD','PLN',
                         'CZK','CNY','MXN','CHF',
                         'KRW','QAR','NZD','ILS',
                         'RUB','IDR','TRY','JPY',
                         'CAD','HKD','ZAR','MYR',
                         'BRL','SGD','INR','DKK',
                         'THB','SEK','NOK','GBP',
                         'EUR','CLP']):
    """Get Historical Daily Exchange rate of currencies to USD"""
    ##import yfinance as yf
    ##import pandas as pd
    if len(list_of_currencies)==1:
        df_currency_ex_rate = yf.download([c+"=X" for c in list_of_currencies],
                                          period='max')[['Adj Close']].rename(
            columns={"Adj Close":list_of_currencies[0]+"=X"})
    else:
        df_currency_ex_rate = yf.download([c+"=X" for c in list_of_currencies],
                                          period='max')['Adj Close']
    ### Melt the currency rates
    df_currency_ex_rate = pd.melt(df_currency_ex_rate.reset_index(),
        id_vars='Date',
        value_name='currency_rate',
        var_name='currency')
    ### Drop missing rates
    df_currency_ex_rate.dropna(subset=['currency_rate'],
                               inplace=True)
    ### trim the last 2 character of Currency Name '=X'
    df_currency_ex_rate["currency"]=df_currency_ex_rate["currency"].apply(lambda x: x[:-2])
    ### Fill in missing values of exchange rate wit previous available rate
    df_currency_ex_rate = df_currency_ex_rate.set_index("Date").groupby(
        ['currency']).resample('1D').ffill().drop(['currency'],axis=1).reset_index()
    df_currency_ex_rate = df_currency_ex_rate.sort_values(['currency','Date'])
    df_currency_ex_rate['source']='yahoo'
    return(df_currency_ex_rate)

In [16]:
def historical_exchage_rate_provider(
    list_of_currencies=['USD','TWD','AUD','PLN',
                         'CZK','CNY','MXN','CHF',
                         'KRW','QAR','NZD','ILS',
                         'RUB','IDR','TRY','JPY',
                         'CAD','HKD','ZAR','MYR',
                         'BRL','SGD','INR','DKK',
                         'THB','SEK','NOK','GBP',
                         'EUR','CLP']):
    """The excel file named currency.xlsx has all the currency and their respective ticker symbol in FRED and Yahoo Finance.
    This functions take those tickers symbols and pull historical currency exchange rate from FED. Then pulls exchange rate from
    Yahoo finance and return aggretated exchange rate from those 2 sources."""
    ##import datetime as dt
    ##import pandas as pd
    ##import pandas_datareader
    ##import yfinance as yf
    ### Get the possible currencies from your data base
    #df_currency = pd.read_excel(path_of_data+"Foreign Exchange Rates/currency.xlsx",sheet_name="currency")
    url_currency = "https://docs.google.com/spreadsheets/d/14t5Yilm6l34AFyb67k3v56R9OpDHZwk8"
    df_currency = pd.read_excel(url_currency+'/export',sheet_name="currency")
    ### Determine which currencies to pull from Fred
    set_of_ticker_currency_to_pull_from_fred = set(df_currency[(df_currency['currency'].isin(list_of_currencies)) &
                                                               (df_currency['source']=='fred')]['ticker'])

    print("Retriving Exchange Rates from FRED")
    df_ex_Rate_fred= pandas_datareader.data.DataReader(set_of_ticker_currency_to_pull_from_fred,
                                                       data_source='fred',
                                                       start=dt.datetime(1950,1,1)).reset_index()
    print("Retriving Exchange Rates from FRED Complete",df_ex_Rate_fred.shape)
    ### Orgonize dataframe
    df_ex_Rate_fred = df_ex_Rate_fred.rename(columns={"DATE":"Date"}).set_index("Date")
    ### Melt the data
    df_ex_Rate_fred = pd.melt(df_ex_Rate_fred.reset_index(),
                                id_vars='Date',
                                value_name='currency_rate',
                                var_name='ticker')
    ### Drop missing rates
    df_ex_Rate_fred.dropna(subset=['currency_rate'],inplace=True)
    ### Resample to have a rate for every possible date in your date range for each currency
    df_ex_Rate_fred = df_ex_Rate_fred.set_index("Date").groupby(['ticker']).resample('1D').ffill().drop(['ticker'],
                                                                                           axis=1).reset_index()
    ### Get the currency standard abbreviation and as well as its target rate
    df_ex_Rate_fred = pd.merge(df_ex_Rate_fred,
                           df_currency[["ticker","toUSD","currency"]],
                           left_on=['ticker'],
                           right_on=['ticker'],
                           how='left')
    ### Some of thses ticker you pulled from the Fred are from 1 Foreign currency to USD.
    ### Most of them are from USD to Foreign Currecny. Make all of them from from USD to Foreign Currecny to keep it consistant with Yahoo Finance
    df_ex_Rate_fred.loc[df_ex_Rate_fred['toUSD']==True,'currency_rate'] = 1.0/df_ex_Rate_fred['currency_rate']
    df_ex_Rate_fred = df_ex_Rate_fred[['currency','Date','currency_rate']]
    df_ex_Rate_fred['source']='fred'

    ####Pull exchange rate from Yhaoo Finance
    print("Retriving Exchange Rates from YahooFinance")
    #print(list_of_currencies)
    df_ex_Rate_yahoo = historical_exchage_rate_provider_from_yahoo_finance(list_of_currencies)
    ### Creata a dataframe for USD to USD exchange rate
    df_ex_rate_usd = pd.DataFrame(pd.date_range(start=dt.datetime(1950,1,1),
                                                end=dt.datetime.now()+ dt.timedelta(days=5),
                                                freq='D'),columns=['Date'])
    df_ex_rate_usd['currency'] = 'USD'
    df_ex_rate_usd['currency_rate'] = 1.00

    ### Concatinate Data from FED and Yahoo Finance and USD Exchange Rate
    df_ex_rate = pd.concat([df_ex_Rate_fred,
                            df_ex_Rate_yahoo,
                            df_ex_rate_usd],
                           ignore_index=True).sort_values(['currency','Date','source'])
    df_ex_rate=df_ex_rate.drop_duplicates(subset=['currency','Date'],
                                          keep='last')
    df_ex_rate.reset_index(drop=True,inplace=True)
    df_ex_rate = df_ex_rate.set_index('Date').groupby(['currency']).resample('1D').ffill().drop(
        ['currency'],axis=1).reset_index()
    df_ex_rate.reset_index(drop=True,inplace=True)
    df_ex_rate.drop(['source'],axis=1,inplace = True)
    return(df_ex_rate)

In [17]:
def yahoo_quotes_provider_via_yfinance(list_of_symbols):
    """Get Current quote from Yahoo Finance for a list of symbols/tciker"""
    print("Number of symbosl to search on yahoo",len(list_of_symbols))
    list_of_df_quotes=[]
    for symbol in list_of_symbols:
        try:
            symbol_obj = yf.Ticker(symbol)
            symbol_info = symbol_obj.get_info()
            df_quote_yahoo_symbol = pd.DataFrame([symbol_info])
            list_of_df_quotes.append(df_quote_yahoo_symbol)
        except:
            pass
    df_quotes = pd.concat(list_of_df_quotes,
                          ignore_index=True)
    df_quotes.rename(columns={"currency":"symbolCurrency"},inplace=True)
    df_quotes['ebit'] = df_quotes['totalRevenue'] * df_quotes['operatingMargins']
    df_quotes['ev'] = (df_quotes['marketCap'] + df_quotes['totalDebt']) - df_quotes['totalCash'] 
    df_quotes['ev_ebit'] = df_quotes['ev']/df_quotes['ebit']
    print(len(df_quotes),"quote from Yfinance")
    return(df_quotes)

In [18]:
def symbol_attribute_retriever_from_yahoo(list_of_symbol):
    ### get histroical exchange rate
    df_ex_rate = historical_exchage_rate_provider()
    df_ex_rate = df_ex_rate[df_ex_rate.groupby(['currency'])['Date'].transform('max') == df_ex_rate['Date']].reset_index(drop=True)
    
    df_place_holder = pd.DataFrame(columns=['symbol','exchange','shortName','industry','sector','heldPercentInsiders','currentPrice','bookValue',
                                        'returnOnAssets','returnOnEquity','firm_to_ebit','priceToBook','trailingPE','forwardPE','dividendYield',
                                            'payoutRatio','fiveYearAvgDividendYield','quickRatio', 'currentRatio','debtToEquity',
                                        'grossMargins','ebitdaMargins','operatingMargins','profitMargins',
                                        'marketCap','ebit','totalCash','totalDebt','totalRevenue','grossProfits','ebitda','operatingCashflow','freeCashflow',
                                        'symbolCurrency','financialCurrency','fullTimeEmployees','longBusinessSummary'])
    df_symbol_attributes_from_yahoo = yahoo_quotes_provider_via_yfinance(list_of_symbol)
    ### pull fudamenetal from yfinance
    df_symbol_attributes_from_yahoo = pd.concat([df_place_holder,df_symbol_attributes_from_yahoo],ignore_index=True)
    ### mereg merge rate with attribuetfb rom yfiance
    df_symbol_attributes_from_yahoo = pd.merge(df_symbol_attributes_from_yahoo,
         df_ex_rate[['currency','currency_rate']].rename(columns={"currency":"symbolCurrency","currency_rate":"currency_rate_symbol"}),
         left_on=['symbolCurrency'],
         right_on=['symbolCurrency'],
         how='left')
    df_symbol_attributes_from_yahoo = pd.merge(df_symbol_attributes_from_yahoo,
             df_ex_rate[['currency','currency_rate']].rename(columns={"currency":"financialCurrency","currency_rate":"currency_rate_financial"}),
             left_on=['financialCurrency'],
             right_on=['financialCurrency'],
             how='left')

    list_of_col_to_convert_to_usd_symbol = ['currentPrice','marketCap']
    list_of_col_to_convert_to_usd_financial = ['totalRevenue','grossProfits','ebitda','ebit','operatingCashflow','bookValue','freeCashflow','totalCash','totalDebt']
    ### Make a copy of what you pulled from Yfinanc and convert them to USD
    df_symbol_attributes_from_yahoo_usd = df_symbol_attributes_from_yahoo.copy()
    df_symbol_attributes_from_yahoo_usd[list_of_col_to_convert_to_usd_financial] = df_symbol_attributes_from_yahoo_usd[list_of_col_to_convert_to_usd_financial].divide(
        df_symbol_attributes_from_yahoo_usd['currency_rate_financial'],axis='index')

    df_symbol_attributes_from_yahoo_usd[list_of_col_to_convert_to_usd_symbol] = df_symbol_attributes_from_yahoo_usd[list_of_col_to_convert_to_usd_symbol].divide(
        df_symbol_attributes_from_yahoo_usd['currency_rate_symbol'],axis='index')
    
    df_symbol_attributes_from_yahoo_usd['firm_to_ebit'] = (df_symbol_attributes_from_yahoo_usd['marketCap']+df_symbol_attributes_from_yahoo_usd['totalDebt'])/ df_symbol_attributes_from_yahoo_usd['ebit']
    ### covnert ratio inn %
    for col in ['returnOnAssets','returnOnEquity','heldPercentInsiders','grossMargins','ebitdaMargins','operatingMargins','profitMargins','dividendYield','payoutRatio']:
        df_symbol_attributes_from_yahoo_usd[col]= df_symbol_attributes_from_yahoo_usd[col]*100
        
    ## convert numebr ijnto MM 
    for col in ['ev','marketCap','totalRevenue','grossProfits','ebitda','ebit','operatingCashflow','freeCashflow','totalCash','totalDebt']:
        df_symbol_attributes_from_yahoo_usd[col]= df_symbol_attributes_from_yahoo_usd[col]/1000000
    
    df_symbol_attributes_from_yahoo_usd = df_symbol_attributes_from_yahoo_usd[
        ['symbol','shortName','industry','sector','heldPercentInsiders','currentPrice','bookValue',
         'returnOnAssets','returnOnEquity','ev','ev_ebit','firm_to_ebit','priceToBook','trailingPE','forwardPE',
         'dividendYield','payoutRatio','fiveYearAvgDividendYield','quickRatio', 'currentRatio','debtToEquity',
         'grossMargins','ebitdaMargins','operatingMargins','profitMargins',
         'marketCap','ebit','totalCash','totalDebt','totalRevenue','grossProfits','ebitda','operatingCashflow','freeCashflow',
         'symbolCurrency','financialCurrency','exchange','fullTimeEmployees','longBusinessSummary']].copy()
    return(df_symbol_attributes_from_yahoo_usd)

In [19]:
def data_frame_flattener(df_data):
    df=df_data.copy()
    try:
        df.columns=[' '.join(map(str,col)).strip() for col in df.columns.values]
    except:
        pass
    return(df)

def column_suffix_adder(df_data,
                        list_of_columns_to_add_suffix_on,
                        suffix):
    """Add specific siffix to specific columns"""
    df=df_data.copy()
    ### Add suffix or prefix to certain columns rename all columns
    new_names = [(i,i+suffix) for i in df[list_of_columns_to_add_suffix_on].columns.values]
    df.rename(columns = dict(new_names), inplace=True)
    return(df)

### Visualization 

In [20]:
def plotly_line_dash_bar_chart(df_data,
                               x,
                               ybar_left=[],
                               ybar_right=[],
                               yline_left=[],
                               yline_right=[],
                               ydash_left=[],
                               ydash_right=[],
                               height=500, 
                               width=1600,
                               rangemode=None,
                               title=None,
                               barmode='group',
                               texttemplate= "%{value}"
                               ):
    """ Graph 2 time series on 2 different y-axis"""
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    #fig.update_yaxes(rangemode='tozero')

    for bar_var in ybar_left:
        # Add traces
        fig.add_trace(
            go.Bar(x=df_data[x],
                   y=df_data[bar_var],
                   name=bar_var,
                   text = df_data[bar_var],
                   textposition="inside",
                   texttemplate= texttemplate,
                   textfont_color="white"),
            secondary_y=False,
            )
    for bar_var in ybar_right:
        # Add traces
        fig.add_trace(
            go.Bar(x=df_data[x],
                   y=df_data[bar_var],
                   name=bar_var,
                   text = df_data[bar_var],
                   textposition="inside",
                   texttemplate= texttemplate,
                   textfont_color="white"),
            secondary_y=True,
            )
    
    for line_var in yline_left:
        fig.add_trace(
            go.Scatter(x=df_data[x], 
                       y=df_data[line_var],
                       name=line_var,
                       #hoverinfo='text',
                       ),
            secondary_y=False,
            )
    
    for line_var in yline_right:
        fig.add_trace(
            go.Scatter(x=df_data[x], 
                       y=df_data[line_var],
                       name=line_var
                       ),
            secondary_y=True,
            )
        
    for dash_var in ydash_left:
        fig.add_trace(
            go.Scatter(x=df_data[x],
                       y=df_data[dash_var],
                       name=dash_var,
                       line = dict(dash='dot')),
            secondary_y=False,
            )
    for dash_var in ydash_right:
        fig.add_trace(
            go.Scatter(x=df_data[x],
                       y=df_data[dash_var],
                       name=dash_var,
                       line = dict(dash='dot')),
            secondary_y=True,
            )
    if rangemode != None:
        fig.update_yaxes(rangemode=rangemode)
    fig = fig.update_layout(height=height, 
                            width=width,
                            title=title,
                            barmode=barmode)
    return(fig)

In [21]:
def net_trade_plotter(df_data):
    df_trade_summary = df_data.copy()
    fig = plotly_line_dash_bar_chart(df_trade_summary,
                               x='date',
                                ybar_left=['trade_amount buy','trade_amount sell'],
                                ybar_right=[],
                                yline_left=[],
                                yline_right=['Net'],
                                ydash_left=[],
                                ydash_right=[],
                                height=500,
                                width=1500,
                                rangemode=None,
                                title="Inflow - Outflow",
                                barmode='relative',
                                texttemplate='%{y:,.0f}').update_layout(hovermode='x')

    left_min = pd.melt(df_trade_summary,id_vars='date',value_vars=['trade_amount buy','trade_amount sell'])['value'].min()*1.1
    left_max = pd.melt(df_trade_summary,id_vars='date',value_vars=['trade_amount buy','trade_amount sell'])['value'].max()*1.1

    right_min = pd.melt(df_trade_summary,id_vars='date',value_vars=['Net'])['value'].min()*1.1
    right_max =pd.melt(df_trade_summary,id_vars='date',value_vars=['Net'])['value'].max()*1.1
    

    fig.update_layout(yaxis=dict(range=[left_min,left_max]), yaxis2=dict(range=[left_min,left_max]))
    fig.update_yaxes(rangemode ='tozero')
    return(fig)

In [22]:
def portfolio_analytics(res_my_portfolio=None,
                        res_trade_histroy=None):
    #### Pull Data Current Holdings
    if res_my_portfolio == None:
        print("Pulling Portfolio Data")
        res_my_portfolio = my_portfolio()
    if res_trade_histroy == None:
        print("Pulling trade history")
        res_trade_histroy = trade_histroy()
    df_my_current_holding = res_my_portfolio['my_current_holdings'].copy().reset_index().rename(columns={'index':'rank'})
    df_open_orders = res_my_portfolio['open_orders'].copy()
    df_portfolio_hist = res_my_portfolio['portfolio_history'].copy()
    df_my_current_holding['rank'] = df_my_current_holding['rank']+1
    ###
    df_trade_summary = res_trade_histroy['df_trade_summary'].copy()
    fig_inflow_outflow = net_trade_plotter(df_trade_summary.set_index("date").resample("1m").sum().reset_index()[2:])
    ### Timeseries of portfolio
    fig_portfolio_timeseries = plotly_line_dash_bar_chart(df_portfolio_hist[df_portfolio_hist['begins_at']>dt.datetime(2020,8,15)],
                                   x='begins_at',
                                   ybar_left=[],
                                   ybar_right=[],
                                   yline_left=['adjusted_close_equity'],
                                   yline_right=[],
                                   ydash_left=[],
                                   ydash_right=['CumReturn'],
                                   height=500, 
                                   width=1350,
                                   rangemode=None,
                                   title=None,
                                   barmode='group',
                                   texttemplate= "%{value}"
                                   )
    fig_portfolio_timeseries.update_layout(hovermode="x",
                                           legend=dict(
                                               yanchor="top",
                                               y=1.2,
                                               xanchor="left",
                                               x=0))
    fig_portfolio_timeseries.show()
    ### Inflow outflow
    fig_inflow_outflow.show()
    ### Sector
    fig_pie_holding = px.sunburst(df_my_current_holding,
                                  path=['sector', 'symbol'], values='equity',
                hover_data=['industry','percent_change','name','rank'],
                height=620,
                                  title='Investment Excl. Cash')
    fig_pie_holding.update_traces(texttemplate = '%{label}<br>%{percentRoot:.2%}')
    fig_pie_holding.show()
    
    ### Industry
    fig_pie_holding3 = px.sunburst(df_my_current_holding,
                                  path=['industry', 'symbol'], values='equity',
                hover_data=['sector','percent_change','name','rank'],
                height=620,
                                  title='Industry | Investment Excl. Cash')
    fig_pie_holding3.update_traces(texttemplate = '%{label}<br>%{percentRoot:.2%}')
    fig_pie_holding3.show()
    
    ### individual Holdings
    fig_pie_holding_2 = px.sunburst(df_my_current_holding,
                                  path=['symbol'], values='equity',
                hover_data=['sector','industry','percent_change','name','rank'],
                height=620,
                                  title='Investment Excl. Cash')
    fig_pie_holding_2.update_traces(texttemplate = '%{label}<br>%{percentRoot:.2%}')
    fig_pie_holding_2.show()
    
    ### Freez 1st row
    df_my_current_holding_formatted = df_my_current_holding.style.set_sticky(axis=1)
    ### format numbers
    df_my_current_holding_formatted = df_my_current_holding_formatted.format({"ROA": "{:,.2f}",
                                                                                "ROE": "{:,.2f}",
                                                                                "quantity": "{:,.0f}",
                                                                                "equity": "{:,.0f}",
                                                                                "equity_initial": "{:,.0f}",
                                                                                "equity_change": "{:,.0f}",
                                                                                'percent_change': "{:,.2f}",
                                                                                'ave_price': "{:,.2f}",
                                                                                "price": "{:,.2f}",
                                                                              'ev': "{:,.2f}",
                                                                                'ev_ebit': "{:,.2f}",
                                                                                "firm_to_ebit": "{:,.2f}",
                                                                                "pe": "{:,.2f}",
                                                                                "fpe": "{:,.2f}",
                                                                                "pb": "{:,.2f}",
                                                                                "pb_ratio": "{:,.2f}",
                                                                                "marketCap": "{:,.0f}",
                                                                                "rev": "{:,.0f}",
                                                                                "ebit": "{:,.0f}",
                                                                                "grossMargins": "{:,.2f}",
                                                                                "operatingMargins": "{:,.2f}",
                                                                                "profitMargins": "{:,.2f}",
                                                                                "totalCash": "{:,.0f}",
                                                                                "totalDebt": "{:,.0f}",
                                                                                "currentRatio": "{:,.2f}",
                                                                                "quickRatio": "{:,.2f}",
                                                                                "grossProfits": "{:,.2f}",
                                                                                "ebitda": "{:,.0f}",
                                                                                "ocf": "{:,.0f}",
                                                                                "ebitdaMargins": "{:,.2f}",
                                                                                "freeCashflow": "{:,.0f}",
                                                                                "heldPercentInsiders": "{:,.2f}",
                                                                                "div_yeild": "{:,.2f}",
                                                                                 "5y_div_yeild": "{:,.2f}",
                                                                                 "payoutRatio": "{:,.2f}",
                                                                                "fullTimeEmployees": "{:,.0f}",
                                                                                'debtToEquity': "{:,.2f}"})
    ### Color Code columns
    from  matplotlib.colors import LinearSegmentedColormap
    c = ["darkred","red","lightcoral","white", "palegreen","green","darkgreen"]
    v = [0,.15,.4,.5,0.6,.9,1.]
    l = list(zip(v,c))
    cmap=LinearSegmentedColormap.from_list('rg',l, N=256)
    
    c2 = ["darkred","lightcoral","white","white","white","lightcoral","darkred"]
    v2 = [0,.2,.4,.5,0.6,.8,1.]
    l2 = list(zip(v2,c2))
    cmap2 =LinearSegmentedColormap.from_list('range',l2, N=256)
    
    df_my_current_holding_formatted.background_gradient(cmap=cmap2, subset=['percentage'])
    df_my_current_holding_formatted.background_gradient(vmin=-100,vmax=100,cmap=cmap, subset=['percent_change'])
    df_my_current_holding_formatted.background_gradient(cmap=cmap, subset=['equity_change'])
    df_my_current_holding_formatted.background_gradient(cmap='Blues', subset=['equity'])
    #df_my_current_holding_formatted.background_gradient(cmap='Blues', subset=['percentage'])
    df_my_current_holding_formatted.background_gradient(cmap='Blues', subset=['CumPercent'])
    display(df_my_current_holding_formatted)
    display(df_open_orders)